# 3. 基本库的使用

## 3.1 urllib

urllib 是 Python 内置的 HTTP 请求库，包含四个模块：
* request: 它是最基本的 HTTP 请求模块，可以用来模拟发送请求。
* error: 异常处理模块，如果出现请求错误，我们可以捕获这些异常。
* parse: 工具模块，提供许多 URL 处理方法，比如拆分、解析、合并等。
* robotparser: 主要是用来识别网站的 robots.txt 文件，判断哪些网页可以爬，哪些不可以爬。

### 3.1.1 发送请求

使用 urllib 的 request 模块，可以方便地实现请求的发送并得到响应。

#### 1. urlopen()

In [ ]:
import urllib.request

response = urllib.request.urlopen('https://www.python.org')
print(response.read().decode('utf-8'))  # read() 可以得到返回的网页内容
print(type(response))  # <class 'http.client.HTTPResponse'>
print(response.status)  # 200
print(response.getheaders())

也可以给链接传递一些参数，urllib()函数的 API:
```
urllib.request.urlopen(url, data=None, [timeout,]*, cafile=None, capath=None, cadefault=False, context=None)
```
* data 参数：可选，需要使用 bytes() 将参数转化为 bytes 类型。另外，需使用 POST 请求方式。

In [5]:
import urllib.parse
import urllib.request

data = bytes(urllib.parse.urlencode({'word': 'hello'}), encoding='utf-8')
response = urllib.request.urlopen('http://httpbin.org/post', data=data)
print(response.read().decode('utf-8'))

{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {
    "word": "hello"
  }, 
  "headers": {
    "Accept-Encoding": "identity", 
    "Content-Length": "10", 
    "Content-Type": "application/x-www-form-urlencoded", 
    "Host": "httpbin.org", 
    "User-Agent": "Python-urllib/3.7"
  }, 
  "json": null, 
  "origin": "114.240.125.16, 114.240.125.16", 
  "url": "https://httpbin.org/post"
}



可见，传递的参数出现在 form 字段中，这表明是模拟了表单提交的方式，以 POST 方式传输数据。
* timeout 参数：设置超时时间，单位为秒。如果超时无响应，会抛异常。默认使用全局默认时间。

In [ ]:
import urllib.request

response = urllib.request.urlopen('http://httpbin.org/get', timeout=1)

* 其它参数: context 参数，必须是 ssl.SSLContext 类型，用来指定 SSL 设置，cafile, capath 分别指定 CA 证书和它的路径，在请求 HTTPS 链接时有用，cadefault 参数已弃用。

#### 2. Request

如果请求中需要加入 Headers 等信息，可以利用更强大的 Request 类来构建。Request 的构造方法如下：
```
class urllib.request.Request(url, data=None, headers={}, origin_req_host=None, unverifiable=False, method=None)
```
* url：请求 url，必传参数，其它都是可选参数；
* data：如果要传，必须是 bytes 类型，可用 urllib.parse 模块里的 urlencode() 进行编码；
* headers：请求头，可以在构造请求时通过 headers 参数直接构造，也可以通过调用请求实例的 add_header() 方法构造；添加请求头最常用的用法就是通过修改 User-Agent 来伪装浏览器；
* origin_req_host：请求方的 host 名称或 IP 地址；
* unverifiable：表示这个请求是否是无法通过验证的，默认是 False；
* method：请求方法，比如 GET、POST、和 PUT 等。

In [4]:
from urllib import request, parse

url = 'http://httpbin.org/post'
headers = {
    'User-Agent': 'Mozilla/4.0 (compatible; MSIE 5.5; Windows NT)',
    'Host': 'httpbin.org'
}
dict = {
    'name': 'Germey'
}
data = bytes(parse.urlencode(dict), encoding='utf-8')
req = request.Request(url=url, data=data, headers=headers, method='POST')
response = request.urlopen(req)
print(response.read().decode('utf-8'))

{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {
    "name": "Germey"
  }, 
  "headers": {
    "Accept-Encoding": "identity", 
    "Content-Length": "11", 
    "Content-Type": "application/x-www-form-urlencoded", 
    "Host": "httpbin.org", 
    "User-Agent": "Mozilla/4.0 (compatible; MSIE 5.5; Windows NT)"
  }, 
  "json": null, 
  "origin": "114.240.125.16, 114.240.125.16", 
  "url": "https://httpbin.org/post"
}



#### 3. 高级用法 

介绍两种高级类，一是 Handler，可以理解为各种处理器，有处理登陆验证的，有处理 Cookies 的，有处理代理设置的。urllib.request 模块里的 BaseHandler 类是其它 Handler 的父类，它提供了最基本的用法，例如 default_open()、protocol_request() 等。一些子类如下：
* HTTPDefaultErrorHandler: 用于处理 HTTP 响应错误；
* HTTPRedirectHandler: 用于处理重定向；
* HTTPCookieProcessor: 用于处理 Cookies；
* ProxyHandler: 用于设置代理，默认代理为空；
* HTTPPasswordMgr: 用于管理密码，它维护了用户名和密码的表；
* HTTPBasicAuthHandler: 用于管理认证，如果一个链接打开时需要认证，那么可以用它来解决认证问题。

另一个就是 OpenerDirector，可以称为 Opener。Opener 可以使用 open() 方法，和 urlopen() 一致。可以利用 Handler 来构建 Opener。

* 验证

有些网站需要输入用户名和密码，可以使用 HTTPBasicAuthHandler 来完成。

In [ ]:
from urllib.request import HTTPPasswordMgrWithDefaultRealm, HTTPBasicAuthHandler, build_opener
from urllib.error import URLError

username = 'username'
password = 'password'
url = 'http://localhost:5000/'

p = HTTPPasswordMgrWithDefaultRealm()
p.add_password(None, url, username, password)
auth_handler = HTTPBasicAuthHandler(p)
opener = build_opener(auth_handler)

try:
    result = opener.open(url)
    html = result.read().decode('utf-8')
    print(html)
except URLError as e:
    print(e.reason)

* 代理

In [ ]:
from urllib.error import URLError
from urllib.request import ProxyHandler, build_opener

proxy_handler = ProxyHandler({
    'http': 'http://127.0.0.1:9743',
    'https': 'http://127.0.0.1:9743'
})
opener = build_opener(proxy_handler)
try:
    response = opener.open('http://baidu.com')
    print(response.read().decode('utf-8'))
except URLError as e:
    print(e.reason)

* Cookies

In [ ]:
import http.cookiejar, urllib.request

cookie = http.cookiejar.CookieJar()
handler = urllib.request.HTTPCookieProcessor(cookie)
opener = urllib.request.build_opener(handler)
response = opener.open('http://www.baidu.com')

### 3.1.2 处理异常

比较常用的两个异常类有：
* URLError: 来自 urllib 的 error 模块，继承自 OSError 类，有一个 reason 属性；
* HTTPError: 是 URLError 的子类，专门来处理 HTTP 请求错误，有三个属性：1）code，返回 HTTP 状态码；2）reason，同父类一样，返回错误的原因；3）headers，返回请求头。

### 3.1.3 解析链接

#### 1. urlparse()

该方法可以实现 URL 的识别和分段：

In [7]:
from urllib.parse import urlparse

result = urlparse('http://baidu.com/index.html;user?id=5#comment')
print(type(result), result)

<class 'urllib.parse.ParseResult'> ParseResult(scheme='http', netloc='baidu.com', path='/index.html', params='user', query='id=5', fragment='comment')


对于 url http://baidu.com/index.html;user?id=5#comment, urlparse() 将其分成了6部分，解析时有特定的分隔符。比如，:// 前面是 scheme，代表协议；第一个 / 符号前面是 netloc，即域名，后面是 path，即访问路径；分号;后面是 params，代表参数；问号?后面是查询条件 query，一般用作 GET 类型的 URL；井号 # 后面是锚点，用于直接定位页面内部的下来位置。所以，一个标准的链接合适为：scheme://netloc/path;params?query#fragment。

#### 2. urlunparse()

参数是一个可迭代对象，长度必须是 6，否则会抛出参数数量不足或过多的问题。

In [8]:
from urllib.parse import urlunparse

data = ['http', 'www.baidu.com', 'index.html', 'user', 'a=6', 'comment']
print(urlunparse(data))

http://www.baidu.com/index.html;user?a=6#comment


其它方法，有 urlsplit()，同 urlparse()，该方法不单独解析 params 这一部分，只返回 5 个结果；urlunsplit() 同 urlunparse()，区别是长度必须是 5；urljoin()，可以实现链接的解析、拼合和生成。urlencode() 将字典数据序列化为 GET 请求参数，比如：

In [9]:
from urllib.parse import urlencode

params = {
    'name': 'germey',
    'age': 22
}
base_url = 'http://www.baidu.com'
url = base_url + urlencode(params)
print(url)

http://www.baidu.comname=germey&age=22


与此相反的是反序列化函数 parse_qs()，可以将 GET 请求参数转化为字典，parse_qsl() 可以将参数转为化元组组成的列表。还有一个重要的函数，quote()，可以将内容转化为 URL 编码的格式。URL 中有中文时，有时可能会导致乱码，可以用该方法转化为 URL 编码：

In [12]:
from urllib.parse import quote

keyword = '壁纸'
url = 'https://www.baidu.com/s?wd=' + quote(keyword)
print(url)

https://www.baidu.com/s?wd=%E5%A3%81%E7%BA%B8


与此相反，unquote() 可以进行 URL 解码。

### 3.1.3 分析 Robots 协议

#### 1. Robots 协议

Robots 协议也称为爬虫协议、机器人协议，用来告诉爬虫和搜索引擎哪些页面可以抓取，哪些不可以抓取。它通常是一个 robots.txt 的文本文件，一般放在网站的根目录下。一个 robots.txt 的样例为：
```
User-agent:*
Disallow: /
Allow: /public/
```

#### 2. robotparser

urllib.robotparse 模块提供了一个类 RobotFileParser，它可以根据某网站的 robots.txt 文件来判断一个爬取爬虫是否有权限来爬取这个网页。该类声明为：
```
urllib.robotparse.RobotFileParser(url='')
```
常用方法为：
* ser_url(): 设置 robots.txt 文件的链接，如果创建 RobotFileParser 对象时传入了链接，则不需要再设；
* read(): 读取 robots.txt 文件并进行分析，该方法一定要调用，不然接下来的判断都为 False；
* parse(): 用来解析 robots.txt 文件；
* can_fetch(): 该方法传入两个参数，第一个是 User-agent，第二个是要抓取的 URL，返回 True 或 False；
* mtime(): 返回上次抓取和分析 robots.txt 的时间；
* modified(): 将当前时间设置为上次抓取和分析 robots.txt 的时间

In [16]:
from urllib.robotparser import RobotFileParser

rp = RobotFileParser()
rp.set_url('http://www.jianshu.com/robots.txt')
rp.read()
print(rp.can_fetch('*', 'http://www.jianshu.com/p/b67554025d7d'))

False


## 3.2 使用 requents

urllib 其实不太好用，比如处理网页验证和 Cookies 时需要写 Opener 和 Handler 来处理，最好使用更强大的库 requests。

### 3.2.1 基本用法

urllib 库中的 urlopen() 方法实际上是以 GET 方式请求网页，而 requests 中相应的方法就是 get():

In [20]:
import requests

r = requests.get('https://www.baidu.com')
print(type(r))
print(r.status_code)
print(type(r.text))
print(r.cookies)

<class 'requests.models.Response'>
200
<class 'str'>
<RequestsCookieJar[<Cookie BDORZ=27315 for .baidu.com/>]>


其它请求依然可以用一行代码来完成：

In [ ]:
r = requests.post('http://httpbin.org/post')
r = requests.put('http://httpbin.org/put')
r = requests.delete('http://httpbin.org/delete')
r = requests.head('http://httpbin.org/get')
r = requests.options('http://httpbin.org/get')

#### 1. GET 请求

详细了解下用 requests 构建 GET 请求。

如果请求中附加额外信息的话，用 params 这个参数：

In [21]:
import requests

data = {
    'name': 'germey',
    'age': 22
}
r = requests.get('http://httpbin.org/get', params=data)
print(r.text)

{
  "args": {
    "age": "22", 
    "name": "germey"
  }, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.22.0"
  }, 
  "origin": "114.244.76.172, 114.244.76.172", 
  "url": "https://httpbin.org/get?name=germey&age=22"
}



可以看到，请求的链接自动被构造成了：https://httpbin.org/get?name=germey&age=22 。

另外，网页返回的结果是 JSON 格式的字符串，可以直接调用 json() 得到字典格式。但需要注意，如果返回的结果不是 JSON 格式，会抛出 json.decoder.JSONDecoderError。

**抓取二进制数据**

In [ ]:
import requests

r = requests.get('https://github.com/favicon.ico')
with open('favicon.ico', 'wb') as f:
    f.write(r.content)

**添加 headers**

通过 headers 参数来传递头信息。

In [ ]:
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.87 Safari/537.36'
}
r = requests.get('https://www.zhihu.com/explore', headers=headers)
print(r.text)

#### 2. Post 请求

In [23]:
import requests

data = {'data': 'germey', 'age': '22'}
r = requests.post('http://httpbin.org/post', data=data)
print(r.text)

{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {
    "age": "22", 
    "data": "germey"
  }, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Content-Length": "18", 
    "Content-Type": "application/x-www-form-urlencoded", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.22.0"
  }, 
  "json": null, 
  "origin": "114.244.76.172, 114.244.76.172", 
  "url": "https://httpbin.org/post"
}



#### 3. 响应

In [ ]:
import requests

r = requests.get('http://www.jianshu.com')
print(r.status_code)
print(r.headers)
print(r.cookies)
print(r.url)
print(r.history)

### 3.2.2 高级用法

#### 1. 文件上传

In [ ]:
import requests

files = {'file': open('favicon.ico', 'rb')}
r = requests.post('http://httpbin.org/post', files=files)
print(r.text)

#### 2. Cookies

前面 urllib 处理过 Cookies，写法比较复杂，而有了 requests，获取和设置 Cookies 只需一步即可完成。

In [ ]:
import requests

r = requests.get('https://www.baidu.com')
print(r.cookies)
for key, value in r.cookies.items():
    print('{}={}'.format(key, value))

可以直接用 Cookie 来维持登陆状态。以知乎为例，登陆知乎后，将网页中 Headers 中的 Cookie 内容复制下来，设置到 Headers 里面，然后发送请求。示例如下：

In [ ]:
import requests

headers = {
    'Cookie': '***',
    'Host': 'www.zhihu.com',
    'User-Agent': '***'
}
r = requests.get('https://www.zhihu.com', headers=headers)
print(r.text)

#### 3. 会话维持

可以利用 Session 对象来维持一个会话。

In [27]:
import requests

s = requests.Session()
s.get('http://httpbin.org/cookies/set/num/123456789')
r = s.get('http://httpbin.org/cookies')
print(r.text)

{
  "cookies": {
    "num": "123456789"
  }
}



#### 4. SSL 证书验证

当发送 HTTP 请求的时候，会检查 SSL 证书，可以使用 verify 参数控制是否检查此证书。如果不加 verify 参数的话，默认是 True，会自动验证。

In [ ]:
import requests

response = requests.get('http://www.12306.cn', verify=False)
print(response.status_code)

#### 5. 代理设置

对于大规模且频繁的请求，网站可能会弹出验证码，或跳转到登陆验证页面，为了防止这种情况，我们需要设置代理来解决这个问题，这就需要用到 proxies 参数。可以用这样的方式设置：

In [ ]:
import requests

proxies = {
    'http': 'http://10.10.1.10:3128',
    'https': 'http://10.10.1.10:1080'
}
requests.get('https://www.taobao.com', proxies=proxies)

#### 6. 超时设置

In [ ]:
import requests

r = requests.get('https://www.taobao.com', timeout=1)
print(r.status_code)

#### 7. 身份认证

In [ ]:
import requests
from requests.auth import HTTPBasicAuth

r = requests.get('http://localhost:5000', auth=HTTPBasicAuth('username', 'password'))
print(r.status_code)

## 3.3 抓取猫眼电影排行

In [ ]:
import json
import requests
import re
import time
from requests.exceptions import RequestException


def get_one_page(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.87 Safari/537.36'
    }
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response.text
        return None
    except RequestException:
        return None

    
def parse_one_page(html):
    pattern = re.compile('<dd>.*?board-index.*?>(\d+)</i>.*?data-src="(.*?)".*?name"><a'
                         +'.*?>(.*?)</a>.*?star">(.*?)</p>.*?releasetime">(.*?)</p>'
                         +'.*?integer">(.*?)</i>.*?fraction">(.*?)</i>.*?</dd>', re.S)
    items = re.findall(pattern, html)
    for item in items:
        yield {
            'index': item[0],
            'image': item[1],
            'title': item[2],
            'actor': item[3].strip()[3:],
            'time': item[4].strip()[5:],
            'score': item[5] + item[6]
        }

    
def write_to_file(content):
    with open('movies.txt', 'r', encoding='utf-8') as f:
        f.write(json.dumps(content, ensure_ascii=False) + '\n')


def main(offset):
    url = 'http://maoyan.com/board/4?offset=' + str(offset)
    html = get_one_page(url)
    for item in parse_one_page(html):
        print(item)
        write_to_file(item)

    
if __name__ == '__main__':
    for i in range(10):
        main(offset=i * 10)
        time.sleep(1)